# Early stage diabetes risk prediction using fast_ai
we have used tabular model with fastai.
here the dataset link:https://archive.ics.uci.edu/dataset/529/early+stage+diabetes+risk+prediction+dataset

In [3]:
#!pip install -Uqq fastbook
#import fastbook
#fastbook.setup_book()

In [6]:
#from fastbook import *
from fastai.tabular.all import *

path=Path('C:/Users/sajib/JupiterNotebook/Ml/Data science with ML/Early stage diabetes risk prediction')
path

Path('C:/Users/sajib/JupiterNotebook/Ml/Data science with ML/Early stage diabetes risk prediction')

In [8]:
df=pd.read_csv(path/'diabetes_data_upload.csv', low_memory=False)

In [9]:
df.columns

Index(['Age', 'Gender', 'Polyuria', 'Polydipsia', 'sudden weight loss',
       'weakness', 'Polyphagia', 'Genital thrush', 'visual blurring',
       'Itching', 'Irritability', 'delayed healing', 'partial paresis',
       'muscle stiffness', 'Alopecia', 'Obesity', 'class'],
      dtype='object')

In [10]:
df.head()

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive


In [11]:
df.tail()

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
515,39,Female,Yes,Yes,Yes,No,Yes,No,No,Yes,No,Yes,Yes,No,No,No,Positive
516,48,Female,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes,No,No,No,Positive
517,58,Female,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,Yes,No,Yes,Positive
518,32,Female,No,No,No,Yes,No,No,Yes,Yes,No,Yes,No,No,Yes,No,Negative
519,42,Male,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Negative


In [13]:
df['Gender'].unique()

array(['Male', 'Female'], dtype=object)

In [14]:
df['class'].unique()

array(['Positive', 'Negative'], dtype=object)

In [15]:
df['Age'].unique()

array([40, 58, 41, 45, 60, 55, 57, 66, 67, 70, 44, 38, 35, 61, 54, 43, 62,
       39, 48, 32, 42, 52, 53, 37, 49, 63, 30, 50, 46, 36, 51, 59, 65, 25,
       47, 28, 68, 56, 31, 85, 90, 72, 69, 79, 34, 16, 33, 64, 27, 29, 26],
      dtype=int64)

Categorify is a TabularProc that replaces a column with a numeric categorical column. FillMissing is a TabularProc that replaces missing values with the median of the column, and creates a new Boolean column that is set to True for any row where the value was missing. These two transforms are needed for nearly every tabular dataset you will use, so this is a good starting point for your data processing:

In [16]:
procs = [Categorify, FillMissing, Normalize]

In [ ]:
#df['Gender'] = df['Gender'].astype('category')
#df['Gender'].cat.set_categories(df['Gender'].unique(), ordered=False, inplace=True)

In [18]:
#https://docs.fast.ai/tutorial.tabular.html
dls=TabularDataLoaders.from_csv(path/'diabetes_data_upload.csv', path=path,
                                y_names='class', #label column
                               cat_names = ['Gender','Polyuria','Polydipsia','sudden weight loss','weakness','Polyphagia','Genital thrush','visual blurring','Itching','Irritability','delayed healing','partial paresis','muscle stiffness','Alopecia','Obesity'], #category columns
                               cont_names=['Age'],#continuous columns
                               procs=procs)

#If you get error message: TypeError: Categorical cannot perform the operation median,
#it is most likely that you have one categorical variable flagged as continuous.
#So when it tried to find its median value during FillMissing, it throws an error (sgugger,forum.fast.ai, 2018).

In [19]:
#The show_batch method works like for every other application:
dls.show_batch()

,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,Age,class
0,Male,No,No,No,Yes,No,No,No,Yes,No,Yes,No,No,Yes,No,46.000000,Negative
1,Female,No,No,No,Yes,No,Yes,No,Yes,No,Yes,Yes,No,Yes,No,55.000000,Negative
2,Male,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,No,45.000000,Negative
3,Female,Yes,Yes,No,Yes,No,No,Yes,No,No,No,No,No,No,No,35.000000,Positive
4,Female,Yes,Yes,Yes,Yes,No,No,Yes,No,No,No,Yes,No,No,Yes,61.999999,Positive
5,Male,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,67.000000,Negative
6,Male,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,31.999999,Negative
7,Male,No,No,No,Yes,Yes,No,Yes,Yes,Yes,Yes,No,Yes,Yes,No,64.000001,Negative
8,Male,No,No,No,No,No,No,No,No,No,No,No,No,No,No,27.000000,Negative
9,Male,Yes,No,No,No,No,No,Yes,No,No,No,Yes,No,Yes,No,60.000000,Positive


In [ ]:
#splits=RandomSplitter(valid_pct=0.2)

In [20]:
dls.xs.iloc[:2]

,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,Age
154,2,2,1,1,2,1,2,2,2,2,1,2,2,2,2,1.153497
218,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1.489875


In [21]:
#We can define a model using the tabular_learner method. When we define our model, fastai will try to infer the loss function based on our y_names earlier.
learn=tabular_learner(dls, metrics=accuracy)

In [22]:
#And we can train that model with the fit_one_cycle method (the fine_tune method won't be useful here since we don't have a pretrained model).
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.710971,0.685204,0.625000,00:00


In [23]:
#We can then have a look at some predictions:
learn.show_results()

,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,Age,class,class_pred
0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,0.575259,1.0,1.0
1,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,-1.489875,1.0,1.0
2,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.572481,0.0,1.0
3,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,-0.085584,1.0,1.0
4,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.153497,1.0,1.0
5,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,0.162232,1.0,1.0
6,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.076848,1.0,1.0
7,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.483918,1.0,1.0
8,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,-0.416005,0.0,1.0


In [24]:
#Or use the predict method on a row:
row, clas, probs = learn.predict(df.iloc[0])

In [25]:
row.show()

,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,Age,class
0,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,40.0,Positive


In [26]:
clas, probs

(tensor(1), tensor([0.4876, 0.5124]))

In [27]:
#To get prediction on a new dataframe, you can use the test_dl method of the DataLoaders. 
#That dataframe does not need to have the dependent variable in its column.
test_df=df.copy()
test_df.drop(['class'], axis=1, inplace=True)

dl=learn.dls.test_dl(test_df)

In [28]:
#Then Learner.get_preds will give you the predictions:
learn.get_preds(dl=dl)

(tensor([[0.4876, 0.5124],
         [0.4808, 0.5192],
         [0.4867, 0.5133],
         ...,
         [0.4804, 0.5196],
         [0.4933, 0.5067],
         [0.4861, 0.5139]]),
 None)

In [38]:
#training more cycles
learn_2 = tabular_learner(dls, metrics=accuracy)
learn_2.fit_one_cycle(20) # using 16th cycle

epoch,train_loss,valid_loss,accuracy,time
0,0.711605,0.732393,0.365385,00:00
1,0.660296,0.755425,0.365385,00:00
2,0.587691,0.763474,0.365385,00:00
3,0.519714,0.756997,0.365385,00:00
4,0.474465,0.727047,0.365385,00:00
5,0.434448,0.674070,0.432692,00:00
6,0.393561,0.631627,0.644231,00:00
7,0.357788,0.575039,0.798077,00:00
8,0.324004,0.486495,0.903846,00:00
9,0.292291,0.416580,0.942308,00:00


In [39]:
learn_2.show_results()

,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,Age,class,class_pred
0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,0.492654,0.0,0.0
1,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.829032,0.0,0.0
2,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,0.740470,1.0,1.0
3,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.740470,0.0,0.0
4,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,3.466447,1.0,1.0
5,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.979550,1.0,1.0
6,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.318707,0.0,0.0
7,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,0.657864,0.0,0.0
8,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,0.823075,0.0,0.0


In [40]:
#Or use the predict method on a row:
row, clas, probs = learn_2.predict(df.iloc[2])

In [41]:
row.show()

,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,Age,class
0,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,41.0,Positive


# Conclusion
we get the accuracy of tabular model 96.15% .